## 1. Importing Libraries

In [ ]:
import logging
import os
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import uncertainty_wizard as uwiz
import matplotlib.pyplot as plt
from uncertainty_wizard.models.stochastic_utils.layers import UwizBernoulliDropout, UwizGaussianDropout, UwizGaussianNoise
from uncertainty_wizard.models._stochastic._stochastic_mode import StochasticMode
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
import seaborn as sns

## Check GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'Number og GPUs Available: {len(gpus)}')
    for gpu in gpus:
        print('Name:', gpu.name, 'Type:', gpu.device_type)
else:
    print('No GPUs found. Please check your TensorFlow installation and GPU setup')

## 2. Load and preprocess MNIST

In [ ]:
# Load mnist data 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Lenght of training samples
print('Lenght of training samples: ', len(x_train))

# Lenght of test samples
print('\nLenght of test samples: ', len(x_test))

# Shape 
print('\nShape: ', x_train[0].shape)
print('\n-----------------------------------------------------------------')

# Preprocess the data
x_train = (x_train.astype('float32') / 255).reshape(x_train.shape[0], 28, 28, 1)
x_test = (x_test.astype('float32') / 255).reshape(x_test.shape[0], 28, 28, 1)


y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

model = uwiz.models.StochasticSequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(UwizBernoulliDropout(0.5, stochastic_mode=StochasticMode()))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.legacy.Adadelta(),
                metrics=['accuracy'])

model.summary()
plt.matshow(x_train[0])
plt.matshow(x_test[0])

## 3. Train

In [ ]:
history = model.fit(x_train,
                    y_train,
                    validation_split=0.1,
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2),
                               tf.keras.callbacks.TensorBoard(log_dir='data/logs', histogram_freq=1)])

# Save the model
model.inner.save_weights('./data/model/mnist_model_stochastic_weights.h5')
print("\nTraining completed, model weights saved")

## 4. Plot Traning & Validation Accuracy and Loss 

In [ ]:
# Plot training & validation accuracy and loss
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()

## 5. Evaluate Model

In [ ]:
# Load the saved model weights
model.inner.load_weights('./data/model/mnist_model_stochastic_weights.h5')
print("Model weights loaded from disk")

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy', test_acc)
print('\nTest loss', test_loss)

# Making predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate the mean squared error
mse = tf.reduce_mean(tf.square(y_pred - y_test))
print(f'\nMSE: {mse:.4f}')


num_samples = 25
predictions = model.predict(x_test[:num_samples])
predicted_labels = np.argmax(predictions, axis=1)

## 6. Adversarial

#### Cleverhans

https://github.com/cleverhans-lab/cleverhans

https://arxiv.org/pdf/1611.01236.pdf


**SparseCategoricalAccuray** is a way to check how often a model correctly predicts which category (class) each data point belongs to. It's "sparse" because the true categories are given as a single numbers like (0,1,2), not lists or vectors.

**SparseCategoricalCrossentropy** measures how well the model's predicted probabilities match the actual categories. It's like a score for the model's errors; the model tries to get this score as low as possible during training to make better predictions.



https://colab.research.google.com/github/andantillon/cleverhans/blob/master/tutorials/future/tf2/notebook_tutorials/mnist_fgsm_tutorial.ipynb#scrollTo=O4teWeVYvUGQ

In [ ]:
if y_test.ndim > 1:
    y_test = np.argmax(y_test, axis=1)

test_accuracy = SparseCategoricalAccuracy()

# Evaluate on clean examples
for x_batch, y_batch in tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128):
    predictions = model.predict(x_batch)
    test_accuracy.update_state(y_batch, predictions)
    
accuracy_clean = test_accuracy.result().numpy() * 100
print(f'\nTest accuracy on clean examples: {accuracy_clean}')

# Adversarial evaluation settings
eps = 0.3
test_accuracy_fgsm = SparseCategoricalAccuracy()
test_accuracy_pgd = SparseCategoricalAccuracy()

# Evaluate on adversarial examples (FGSM and PGD)
for x_batch, y_batch in tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128):
    # FGSM examples
    x_adv_fgsm = fast_gradient_method(model.inner, x_batch, eps, np.inf)
    predictions_fgsm = model.predict(x_adv_fgsm)
    test_accuracy_fgsm.update_state(y_batch, predictions_fgsm)
    
    # PGD examples
    x_adv_pdg = projected_gradient_descent(model.inner, x_batch, eps, 0.01, 40, np.inf)
    predictions_pdg = model.predict(x_adv_pdg)
    test_accuracy_pgd.update_state(y_batch, predictions_pdg)
    
accuracy_fgsm = test_accuracy_fgsm.result().numpy() * 100
accuracy_pdg = test_accuracy_pgd.result().numpy() * 100

print(f'\nTest accuracy on FGSM adversarial examples: {accuracy_fgsm}')
print(f'Test accuracy on PDG adversarial examples: {accuracy_pdg}')

accuracies = [accuracy_clean, accuracy_fgsm, accuracy_pdg]

## 7. Plot Predictions, Confusion Matrix, Adversarial examples and Comparison between

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(num_samples):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title(f'True: {y_true[i]}, Predicted: {predicted_labels[i]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues',
            xticklabels=[str(i) for i in range(10)],
            yticklabels=[str(i) for i in range(10)])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True Label')
plt.show()

# Clean and Adversarial examples

#x_adv_fgsm = fast_gradient_method(model.inner, x_test[:num_samples], eps, np.inf)
#predictions_clean = np.argmax(model.predict(x_test[:num_samples]), axis=1)
#predictions_adv = np.argmax(model.predict(x_adv_fgsm), axis=1)

# Generate FGSM adversarial examples
x_adv_fgsm = fast_gradient_method(model.inner, x_test[:num_samples], eps, np.inf)
predictions_fgsm = np.argmax(model.predict(x_adv_fgsm), axis=1)

# Generate PGD adversarial examples
x_adv_pgd = projected_gradient_descent(model.inner, x_test[:num_samples], eps, 0.01, 40, np.inf)
predictions_pgd = np.argmax(model.predict(x_adv_pgd), axis=1)

plt.figure(figsize=(20, 10))
#plt.figure(figsize=(2 * num_samples, 6))

for i in range(num_samples):
    # Plot clean images
        plt.subplot(3, num_samples, i + 1)
        plt.imshow(x_test[i], cmap='gray')  # If x_test is not already in 28x28, reshape is needed
        plt.title(f'Clean\nPred: {np.argmax(model.predict(x_test[i:i+1]), axis=1)[0]}', fontsize=9)
        plt.axis('off')

        # Plot FGSM adversarial images
        plt.subplot(3, num_samples, num_samples + i + 1)
        plt.imshow(x_adv_fgsm[i], cmap='gray')
        plt.title(f'FGSM\nPred: {predictions_fgsm[i]}', fontsize=9)
        plt.axis('off')

        # Plot PGD adversarial images
        plt.subplot(3, num_samples, 2 * num_samples + i + 1)
        plt.imshow(x_adv_pgd[i], cmap='gray')
        plt.title(f'PGD\nPred: {predictions_pgd[i]}', fontsize=9)
        plt.axis('off')
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
filename = f'pred_clean_vs_adv_{timestamp}.png'
plot_dir = './data/plots//evaluate'
os.makedirs(plot_dir, exist_ok=True)
plt.savefig(os.path.join(plot_dir, filename))

plt.show()


# Compare
plt.figure(figsize=(8, 6))
bar_positions = np.arange(len(accuracies))
plt.bar(bar_positions, accuracies, color=['blue', 'green', 'red'])
plt.xticks(bar_positions, labels=['Clean', 'FGSM', 'PGD'])
plt.ylabel('Accuracy (%)')
plt.title('Model Accuracy: Clean vs Adversarial Examples')
plt.ylim(0, 110)

for i, acc in enumerate(accuracies):
    plt.text(i, acc + 2, f'{acc:.2f}', ha='center', va='bottom')
    
plt.show()

## 8. Analyze the Uncertainty

In [ ]:
# Load the saved model weights 
model.inner.load_weights('./data/model/mnist_model_stochastic_weights.h5')
print("Model weights loaded from disk")

# Perform uncertainty quantification
quantifiers = ['pcs', 'mean_softmax']
results = model.predict_quantified(x_test,
                                    quantifier=quantifiers,
                                    batch_size=64,
                                    sample_size=32,
                                    verbose=1)


## 9. Plots for Uncertainty Analysis

In [ ]:
# Plot the uncertainty distribution
uncertainty_scores = results[1][1]
plt.figure(figsize=(8, 6))
plt.hist(uncertainty_scores, bins=50, alpha=0.7, color='blue')
plt.title('Distribution of Uncertainty Scores')
plt.xlabel('Uncertainty Score')
plt.ylabel('Frequency')
plt.show()

# Plot the predictive confidence score
confidence_scores = results[1][0]
plt.figure(figsize=(8, 6))
plt.hist(confidence_scores, bins=50, alpha=0.7, color='green')
plt.title('Distribution of Predictive Confidence Scores')
plt.xlabel('Predictive Confidence Score')
plt.ylabel('Frequency')
plt.show()

# Plot the predictive confidence score vs uncertainty score
plt.figure(figsize=(8, 6))
plt.scatter(confidence_scores, uncertainty_scores, alpha=0.5, color='red')
plt.title('Predictive Confidence Score vs Uncertainty Score')
plt.xlabel('Predictive Confidence Score')
plt.ylabel('Uncertainty Score')
plt.show()

# Plot PCS and Mean Softmax scores for a subset of the test set data  
#qualitative, visual correlation between each image and its corresponding scores.
num_samples = 25
pcs_scores, mean_softmax_scores = results[0][1], results[1][1]
for i in range(num_samples):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title(f'PCS: {pcs_scores[i]:.2f}\nMean Softmax: {mean_softmax_scores[i]:.2f}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Plot distribution of PCS and Mean Softmax scores
pcs_scores = results[0][1]
mean_softmax_scores = results[1][1]
plt.subplot(1, 2, 1)
plt.hist(pcs_scores, bins=50, alpha=0.7)
plt.xlabel('PCS Score')
plt.ylabel('Frequency')
plt.title('Distribution of PCS Scores')
plt.subplot(1, 2, 2)
plt.hist(mean_softmax_scores, bins=50, alpha=0.7)
plt.xlabel('Mean Softmax Score')
plt.ylabel('Frequency')
plt.title('Distribution of Mean Softmax Scores')
plt.tight_layout()
plt.show()


In [ ]:
pcs_scores = results[0][1][:num_samples]  
mean_softmax_scores = results[1][1][:num_samples]

# Plot PCS scores
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.bar(range(num_samples), pcs_scores)
plt.xlabel('Sample')
plt.ylabel('PCS Score')
plt.title('PCS Scores for Test Samples')

# Plot Mean Softmax scores
plt.subplot(1, 2, 2)
plt.bar(range(num_samples), mean_softmax_scores)
plt.xlabel('Sample')
plt.ylabel('Mean Softmax Score')
plt.title('Mean Softmax Scores for Test Samples')

plt.tight_layout()
plt.show()

In [ ]:
print(x_test.shape)
pcs_predictions = results[0][0]
pcs_confidences = results[0][1]
mean_softmax_predictions = results[1][0]
mean_softmax_confidences = results[1][1]


plt.hist(pcs_confidences, bins=50, alpha=0.7, color='blue', label='PCS')
plt.hist(mean_softmax_confidences, bins=50, alpha=0.7, color='green', label='Mean Softmax')
plt.title('Distribution of Predictive Confidence Scores')
plt.xlabel('Predictive Confidence Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()



plt.hist(pcs_predictions, bins=50, alpha=0.7, color='blue', label='PCS Predictions')
plt.hist(mean_softmax_predictions, bins=50, alpha=0.7, color='green', label='Mean Softmax Predictions')
plt.title('Distribution of Predictive Confidence Scores')
plt.xlabel('Predictive Confidence Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()
